In [2]:
from timor.Bodies import Body, Connector, Gender
from timor.Joints import Joint
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
import xml.etree.ElementTree as ET
import numpy as np
from timor.Robot import PinRobot, RobotBase
from pathlib import Path
import pinocchio as pin
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere
from timor.utilities.transformation import Transformation

from timor.utilities.spatial import rotX, rotY, rotZ

In [3]:

density = 1200  # kg / m3 estimated overall material density
inner_diameter = 60 / 1000  # assume the links are hollow, with 5cm thick walls

def cylinder_mass(l: float, r: float, r_inner: float = 0) -> float:
    """Calculates the mass of a (hollow) cylinder"""
    mass = density * l * np.pi * r ** 2
    hollow_mass = density * l * np.pi * r_inner ** 2
    return mass - hollow_mass


def cylinder_inertia(l: float, r: float, r_inner: float = 0):
    """Calculates the inertia of a (hollow) cylinder, assuming a centered coordinate system"""
    mass = cylinder_mass(l, r, r_inner)
    lever = np.asarray([0, 0, 0])
    I = np.zeros((3, 3))
    I[0, 0] = (1 / 12) * mass * (3 * (r_inner ** 2 + r ** 2) + l ** 2)
    I[1, 1] = I[0, 0]
    I[2, 2] = .5 * mass * (r_inner ** 2 + r ** 2)

    return pin.Inertia(mass, lever, I)

In [16]:
from timor.Geometry import Mesh



sizes = (150 / 1000, 300 / 1000, 450 / 1000)  # we are going to build links in various lengths (sizes) [m]
diameter = 80 / 1000  # the diameter of our links [m]
ROT_X = Transformation.from_rotation(rotX(np.pi)[:3, :3])  # A shortcut we are going to use frequently
ROT_Y = Transformation.from_rotation(rotY(np.pi / 2)[:3, :3])

stl_path_1 = "./assets/Assem_4310_BASE/Assem_4310_BASE/meshes/base_motor_link.STL"
geometry1 = Mesh({"file": stl_path_1})
stl_path_2 = "./assets/Assem_4310_BASE/Assem_4310_BASE/meshes/base_out_link.STL"
geometry2 = Mesh({"file": stl_path_2})

geo1_connector1 = Connector( connector_id = "geo1_connector_1", #create a unique id for connector
                      parent = None, #the parent body for this connector. will be assigned later if attached to body later
                      gender=Gender.f, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                      connector_type='default', #defines the type of connector. optional
                      size=[diameter]) #defines the "size" of the connector. completely optional
geo1_connector2 = Connector( connector_id = "geo1_connector_2", #create a unique id for connector
                      parent = None, #the parent body for this connector. will be assigned later if attached to body later
                      gender=Gender.m, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                      connector_type='default', #defines the type of connector. optional
                      size=[diameter]) #defines the "size" of the connector. completely optional

geo2_connector1 = Connector( connector_id = "geo2_connector_1", #create a unique id for connector
                      parent = None, #the parent body for this connector. will be assigned later if attached to body later
                      gender=Gender.f, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                      connector_type='default', #defines the type of connector. optional
                      size=[diameter]) #defines the "size" of the connector. completely optional

geo2_connector2 = Connector( connector_id = "geo2_connector_2", #create a unique id for connector
                      parent = None, #the parent body for this connector. will be assigned later if attached to body later
                      gender=Gender.m, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                      connector_type='default', #defines the type of connector. optional
                      size=[diameter]) #defines the "size" of the connector. completely optional
body1 = Body(body_id = "test_body_1", #create a unique id for this component
            collision = geometry1, #the collision hitbox uses a Geometry instance
            visual = geometry1, #specifies the Geometry to use for visualizing. Will default to collision.
            connectors = [geo1_connector1, geo1_connector2], #defines what connectors are on this body
            inertia = cylinder_inertia(sizes[0], diameter / 2, inner_diameter / 2), #define the inertia (pinocchio object)
            in_module = None #define what module this body is in. updated automatically if body is added to module later 
            )
body2 = Body(body_id = "test_body_2", #create a unique id for this component
            collision = geometry2, #the collision hitbox uses a Geometry instance
            visual = geometry2, #specifies the Geometry to use for visualizing. Will default to collision.
            connectors = [geo2_connector1, geo2_connector2], #defines what connectors are on this body
            inertia = cylinder_inertia(sizes[1], diameter / 2, inner_diameter / 2), #define the inertia (pinocchio object)
            in_module = None #define what module this body is in. updated automatically if body is added to module later 
            )

joint = Joint(joint_id = "joint_1", #create a unique id for this component
              parent_body = body1, #every joint links a parent body to a child body
              child_body = body2, #every joint links a parent body to a child body
              in_module = None,
              q_limits = (-np.pi, np.pi), #the numerical range for this joint
              torque_limit= np.inf,
              velocity_limit = np.inf,
              acceleration_limit = np.inf,
              parent2joint = Transformation.neutral(), #transformation between base frame of parent link and this joint 
              joint2child = ROT_Y @ Transformation.from_translation([0, 0, sizes[1] / 2]), #transformation between this joint and child link
              joint_type = "revolute"
              )

In [17]:
#create a module
from datetime import datetime
module_header = ModuleHeader(ID='T-link',
                                 name='T Link Module',
                                 date=datetime(2024, 12, 7),
                                 author=['Calix Tang'],
                                 email=['calix@berkeley.edu'],
                                 affiliation=['UC Berkeley']
                                 )

# https://timor-python.readthedocs.io/en/latest/autoapi/timor/Module/index.html
t_link_module = AtomicModule(module_header, #module header definition
                             bodies = (body1, body2), #add all bodies
                             joints = [joint] #add all joints
                             )

print(t_link_module.available_connectors)

{('T-link', 'test_body_2', 'geo2_connector_2'): <timor.Bodies.Connector object at 0x7fb1eb4453c0>, ('T-link', 'test_body_2', 'geo2_connector_1'): <timor.Bodies.Connector object at 0x7fb1eaffef80>, ('T-link', 'test_body_1', 'geo1_connector_1'): <timor.Bodies.Connector object at 0x7fb227d57ee0>, ('T-link', 'test_body_1', 'geo1_connector_2'): <timor.Bodies.Connector object at 0x7fb1eb474400>}


In [18]:
#Create a ModulesDB - a collection of unique modules. For now, we only have 1.
db = ModulesDB([t_link_module])
db.debug_visualization()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [28]:
from timor.Geometry import Mesh


def create_body_with_stl(name: str, stl_path: str, connector_name: str) -> Body:
    """
    Create a Body object in timor using an STL file for collision and visual geometry.

    Args:
        name (str): Name of the body.
        stl_path (str): Path to the STL file.
        connector_name (str): Name of the connector.

    Returns:
        Body: A Body object with the STL-based geometry.
    """
    # Define the collision and visual geometry using the STL file
    stl_geometry = Mesh({"file": stl_path})



    
    # # Create dummy bodies for demonstration
    diameter = 80 / 1000  # the diameter of our links [m]
    length = 150 / 1000
    # Create the body using the STL geometry
    trans = Transformation.from_translation([0, 0, 0]) 
    body = Body(
        body_id=name,
        collision=stl_geometry,
        connectors=[Connector(f'J2_distal+{name}',
                                        trans,
                                        gender=Gender.m,
                                        connector_type='default',
                                        size=[diameter])],
        inertia=cylinder_inertia(length, diameter / 2, inner_diameter / 2)  # Optionally specify inertia if available
    )

    return body

length = 150 / 1000
# Example usage
parent_stl_file_path = "./assets/Assem_4310_BASE/Assem_4310_BASE/meshes/base_motor_link.STL"
child_stl_file_path = "./assets/Assem_4310_BASE/Assem_4310_BASE/meshes/base_out_link.STL"
parent = create_body_with_stl("base_motor_link", parent_stl_file_path, "base_connector")
child = create_body_with_stl("base_out_link", child_stl_file_path, "out")


joint = Joint(
    joint_id='Revolute',
    joint_type='revolute',
    parent_body=parent,
    child_body=child,
    q_limits=np.array([-np.pi, np.pi]),
    torque_limit=1000,
    acceleration_limit=5,
    velocity_limit=10,
    parent2joint=Transformation.from_roto_translation(R=[0,0,0],
                                       p=[0, 1, 0])
    joint2child=Transformation.from_translation([0, 0.05, 0])
)

    
# Module header
module_header = ModuleHeader(
    ID="R",
    name="RJ",
    date="2024-12-04",
    author=["Generated by URDF to Timor"],
    email=["support@example.com"],
    affiliation=["Automated Conversion"]
)

# Return as a ModulesDB

timor_modules = ModulesDB({AtomicModule(module_header, [parent, child], [joint])})
timor_modules.debug_visualization()
print(timor_modules)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2001984271.py, line 57)